<a href="https://colab.research.google.com/github/nclarknz/DarkNet-Obj-Training/blob/main/YOLOv4_tiny_Darknet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction


In this notebook, we implement the tiny version of [YOLOv4](https://arxiv.org/pdf/2004.10934.pdf) for training on your own dataset, [YOLOv4 tiny](https://github.com/AlexeyAB/darknet/issues/6067). This is where darknet came from

Also recommend reading the roboflow blog post on [Training YOLOv4 on custom data](https://blog.roboflow.ai/training-yolov4-on-a-custom-dataset/) side by side. This is where I got the original file from, but then modified to suit my requirements

We will take the following steps to implement YOLOv4 on our custom data:
* Configure our GPU environment on Google Colab
* Install the Darknet YOLOv4 training environment
* Download our custom dataset for YOLOv4 and set up directories
      Training data iages and yolo files should be in darknet/data/obj 
      Valid data images and yolo file should be in darknet/data/objvalid
      
      change darknet/data/obj.data to suit setup and filenames
      darknet/data/obj.names shoul dbe list of classes / labelnames 

* Configure a custom YOLOv4 training config file for Darknet
* Train our custom YOLOv4 object detector
* Reload YOLOv4 trained weights and make inference on test images

When you are done you will have a custom detector that you can use. 


# Configuring cuDNN on Colab for YOLOv4



In [ ]:
# Setup the Google Drive and stuff
from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/My Drive/KiwiBoxSet
%cd /content
!ln -s  '/content/gdrive/My Drive/KiwiBoxSet' kiwiboxset

In [ ]:
# CUDA: Let's check that Nvidia CUDA drivers are already pre-installed and which version is it.
!/usr/local/cuda/bin/nvcc --version
# We need to install the correct cuDNN according to this output

In [ ]:
#take a look at the kind of GPU we have
!nvidia-smi

# Installing Darknet for YOLOv4 on Colab




In [4]:
%cd /content/kiwiboxset
#%rm -rf darknet

/content/gdrive/My Drive/KiwiBoxSet


In [ ]:
#we clone the fork of darknet maintained by roboflow
#small changes have been made to configure darknet for training
!git clone https://github.com/roboflow-ai/darknet.git

**IMPORTANT! If you're not using a Tesla P100 GPU, then uncomment the sed command and replace the arch and code with that matching your GPU. A list can be found [here](http://arnon.dk/matching-sm-architectures-arch-and-gencode-for-various-nvidia-cards/).**

In [ ]:
#install environment from the Makefile
%cd /content/kiwiboxset/darknet/
# compute_30, sm_30 for Tesla K80
# compute_75, sm_75 for Tesla T4
!sed -i 's/ARCH= -gencode arch=compute_60,code=sm_60/ARCH= -gencode arch=compute_75,code=sm_75/g' Makefile
#!sed -i 's/ARCH= -gencode arch=compute_75,code=sm_75/ARCH= -gencode arch=compute_30,code=sm_30/g' Makefile
#install environment from the Makefile
#note if you are on Colab Pro this works on a P100 GPU
#if you are on Colab free, you may need to change the Makefile for the K80 GPU
#this goes for any GPU, you need to change the Makefile to inform darknet which GPU you are running on.
!make clean
!make

Next Stage will download the tiny weights file for Yolo v4. Use this to start the training. Then can us ethe last weights file in teh bacup folder to continue training.

In [ ]:
#download the newly released yolov4-tiny weights
%cd /content/kiwiboxset/darknet
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29
#!wget https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov4-tiny.cfg

# Set up Custom Dataset for YOLOv4

Move files around from zipped folder and setup the training and validation files if required. Mainly commented out as do this local before upload of the files

In [16]:
#Set up training file directories for custom dataset
%cd /content/kiwiboxset/darknet/data/obj/12.zip\ \(Unzipped\ Files\)/

#%cp train/_darknet.labels data/obj.names
#%mkdir data/obj
#copy image and labels
#%cp train/*.jpg data/obj/
#%cp valid/*.jpg data/obj/
%mv * /content/kiwiboxset/darknet/data/obj
#%cp train/*.txt data/obj/
#%cp valid/*.txt data/obj/
%ls -l /content/kiwiboxset/darknet/data/obj | wc -l
%ls -l  | wc -l
#with open('data/obj.data', 'w') as out:
#  out.write('classes = 26\n')
#  out.write('train = data/train.txt\n')
#  out.write('valid = data/valid.txt\n')
#  out.write('names = data/obj.names\n')
#  out.write('backup = backup/')

#write train file (just the image list)
import os

#with open('data/train.txt', 'w') as out:
#  for img in [f for f in os.listdir('train') if f.endswith('jpg')]:
#    out.write('data/obj/' + img + '\n')

#write the valid file (just the image list)
#import os

#with open('data/valid.txt', 'w') as out:
#  for img in [f for f in os.listdir('valid') if f.endswith('jpg')]:
#    out.write('data/obj/' + img + '\n')

/content/gdrive/My Drive/KiwiBoxSet/darknet/data/obj/12.zip (Unzipped Files)
3548
1


# Write Custom Training Config for YOLOv4

In [8]:
#we build config dynamically based on number of classes
#we build iteratively from base config files. This is the same file shape as cfg/yolo-obj.cfg
def file_len(fname):
  with open(fname) as f:
    for i, l in enumerate(f):
      pass
  return i + 1

num_classes = file_len('data/obj.names')
max_batches = num_classes*5000
steps1 = .8 * max_batches
steps2 = .9 * max_batches
steps_str = str(steps1)+','+str(steps2)
num_filters = (num_classes + 5) * 3


print("writing config for a custom YOLOv4 detector detecting number of classes: " + str(num_classes))

#Instructions from the darknet repo
#change line max_batches to (classes*2000 but not less than number of training images, and not less than 6000), f.e. max_batches=6000 if you train for 3 classes
#change line steps to 80% and 90% of max_batches, f.e. steps=4800,5400
if os.path.exists('./cfg/custom-yolov4-tiny-detector.cfg'): os.remove('./cfg/custom-yolov4-tiny-detector.cfg')


#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

  



writing config for a custom YOLOv4 detector detecting number of classes: 26


In [9]:
%%writetemplate ./cfg/custom-yolov4-tiny-detector.cfg
[net]
# Testing
#batch=1
#subdivisions=1
# Training
batch=64
subdivisions=24
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.00261
burn_in=1000
max_batches = {max_batches}
policy=steps
steps={steps_str}
scales=.1,.1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[route]
layers=-1
groups=2
group_id=1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

[route]
layers = -1,-2

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=leaky

[route]
layers = -6,-1

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[route]
layers=-1
groups=2
group_id=1

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[route]
layers = -1,-2

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

[route]
layers = -6,-1

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

[route]
layers=-1
groups=2
group_id=1

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[route]
layers = -1,-2

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[route]
layers = -6,-1

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

##################################

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters={num_filters}
activation=linear



[yolo]
mask = 3,4,5
anchors = 10,14,  23,27,  37,58,  81,82,  135,169,  344,319
classes={num_classes}
num=6
jitter=.3
scale_x_y = 1.05
cls_normalizer=1.0
iou_normalizer=0.07
iou_loss=ciou
ignore_thresh = .7
truth_thresh = 1
random=0
nms_kind=greedynms
beta_nms=0.6

[route]
layers = -4

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

[upsample]
stride=2

[route]
layers = -1, 23

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters={num_filters}
activation=linear

[yolo]
mask = 1,2,3
anchors = 10,14,  23,27,  37,58,  81,82,  135,169,  344,319
classes={num_classes}
num=6
jitter=.3
scale_x_y = 1.05
cls_normalizer=1.0
iou_normalizer=0.07
iou_loss=ciou
ignore_thresh = .7
truth_thresh = 1
random=0
nms_kind=greedynms
beta_nms=0.6
max=200

In [10]:
#here is the file that was just written. 
#you may consider adjusting certain things

#like the number of subdivisions 64 runs faster but Colab GPU may not be big enough
#if Colab GPU memory is too small, you will need to adjust subdivisions to 16
%cat cfg/custom-yolov4-tiny-detector.cfg

[net]
# Testing
#batch=1
#subdivisions=1
# Training
batch=64
subdivisions=24
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.00261
burn_in=1000
max_batches = 130000
policy=steps
steps=104000.0,117000.0
scales=.1,.1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[route]
layers=-1
groups=2
group_id=1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

[route]
layers = -1,-2

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=leaky

[route]
layers = -6,-1

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky



# Train Custom YOLOv4 Detector

In [ ]:
%cd /content/kiwiboxset/darknet/
!./darknet detector train data/obj.data cfg/custom-yolov4-tiny-detector.cfg yolov4-tiny.conv.29 -dont_show -map
#!./darknet detector train data/obj.data cfg/custom-yolov4-tiny-detector.cfg backup/custom-yolov4-tiny-detector_last.weights -dont_show -map
#If you get CUDA out of memory adjust subdivisions above!
#adjust max batches down for shorter training above

# Infer Custom Objects with Saved YOLOv4 Weights

In [ ]:
%cd /content/kiwiboxset/darknet/

In [ ]:
import os
#define utility function
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
#check if weigths have saved yet
#backup houses the last weights for our detector
#(file yolo-obj_last.weights will be saved to the build\darknet\x64\backup\ for each 100 iterations)
#(file yolo-obj_xxxx.weights will be saved to the build\darknet\x64\backup\ for each 1000 iterations)
#After training is complete - get result yolo-obj_final.weights from path build\darknet\x64\bac
!ls -l backup
#if it is empty you haven't trained for long enough yet, you need to train for at least 100 iterations
#!chmod 775 darknet

In [ ]:
#coco.names is hardcoded somewhere in the detector
%cp data/obj.names data/coco.names

In [ ]:

#/test has images that we can test our detector on
test_images = [f for f in os.listdir('test') if f.endswith('.jpg')]
#import random
#img_path = "test/" + random.choice(test_images);
for img in test_images: 
#test out our detector!
    !./darknet detect cfg/custom-yolov4-tiny-detector.cfg backup/custom-yolov4-tiny-detector_best.weights test/{img} -dont-show
    #imShow('predictions.jpg')

#Now Convert to TF Lite

In [ ]:
%cd /content/kiwiboxset/
!git clone https://github.com/hunglc007/tensorflow-yolov4-tflite.git
%cd /content/kiwiboxset/tensorflow-yolov4-tflite

Then, we'll change the labels from the default COCO to our own custom ones.

In [ ]:
!cp /content/kiwiboxset/darknet/data/obj.names /content/kiwiboxset/tensorflow-yolov4-tflite/data/classes/
!ls /content/kiwiboxset/tensorflow-yolov4-tflite/data/classes/

coco.names  obj.names  voc.names  yymnist.names


In [ ]:
!sed -i "s/coco.names/obj.names/g" /content/kiwiboxset/tensorflow-yolov4-tflite/core/config.py

##Convert to TFLite

Time to convert! We'll convert to both a regular TensorFlow SavedModel and to TensorFlow Lite. For TensorFlow Lite, we'll convert to a different TensorFlow SavedModel beforehand.

In [ ]:
%cd /content/kiwiboxset/tensorflow-yolov4-tflite
# Regular TensorFlow SavedModel
!python save_model.py \
  --weights /content/kiwiboxset/darknet/backup/custom-yolov4-tiny-detector_best.weights \
  --output ./checkpoints/yolov4-tiny-512512v2 \
  --input_size 512 \
  --model yolov4 \
  --tiny \

# SavedModel to convert to TFLite
!python save_model.py \
  --weights /content/kiwiboxset/darknet/backup/custom-yolov4-tiny-detector_best.weights \
  --output ./checkpoints/yolov4-tiny-pretflite-512512v2 \
  --input_size 512 \
  --model yolov4 \
  --tiny \
  --framework tflite

# Convert the TensorFlow weights to TensorFlow Lite

In [ ]:
%cd /content/kiwiboxset/tensorflow-yolov4-tflite
!python convert_tflite.py --weights ./checkpoints/yolov4-tiny-pretflite-512512v2 --output ./checkpoints/yolov4-tiny-512v2.tflite

In [ ]:
!ls /content/kiwiboxset/darknet/test

In [ ]:
# Verify
%cd /content/kiwiboxset/tensorflow-yolov4-tflite
!python detect.py --weights ./checkpoints/yolov4-tiny-1024768v5 --size 1024x768 --model yolov4 \
  --image /content/kiwiboxset/darknet/test/imgma.JPG \
  # --framework tflite

In [ ]:
%cd /content/kiwiboxset/tensorflow-yolov4-tflite/
!ls
from IPython.display import Image
Image('/content/kiwiboxset/tensorflow-yolov4-tflite/result.png')

# Eval Model

Not sure if fully working

In [ ]:
# evaluate yolov4 model
%cd /content/kiwiboxset/tensorflow-yolov4-tflite/
!python evaluate.py --weights /content/kiwiboxset/tensorflow-yolov4-tflite/checkpoints/yolov4-tiny-1024768v5/  --tiny True  --size 1024x768 --model yolov4
#%cd mAP/extra
#python remove_space.py
#%cd ..
#python main.py --output results_yolov4_tf

In [ ]:
%cd /content/kiwiboxset/tensorflow-yolov4-tflite/mAP
#%cd mAP/extra
#!python remove_space.py
#%cd ..
!python main.py --output results_yolov4_tf